In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import datetime
from datetime import date

In [2]:
file=os.path.join('x:/Marphys_Archive','Data','OSNAP','RHADCP','RHADCP_S55_data_JC238','S200044A008_RHADCP_2020','S200044A008_RHADCP_2020_A0.whr')
#  extract the header data
with open(file) as file:
    for line in file:
        print(line.rstrip())

[X:\Marphys_Archive\Data\OSNAP\RHADCP\RHADCP_S55_data_JC238\S200044A008_RHADCP_2020\S200044A008_RHADCP_2020_A0.prf]
---------------------------------------------------------------------
Number of measurements                30863
Number of checksum errors             0
Time of first measurement             15/10/2020 11:30:01.001
Time of last measurement              20/07/2022 10:30:55.001

Hardware configuration
---------------------------------------------------------------------
Instrument name                       Signature55
Serial number                         200044
Head frequency                        55 kHz
Firmware version                      169.2211_4
Pressure sensor                       YES
Compass                               YES
Tilt sensor                           YES
Temperature sensor                    YES
Number of head beams                  3

Deployment configuration
---------------------------------------------------------------------
Sound speed        

In [3]:
file=os.path.join('x:/Marphys_Archive','Data','OSNAP','RHADCP','RHADCP_S55_data_JC238','S200044A008_RHADCP_2020','S200044A008_RHADCP_2020_A0_p.dat')
file

'x:/Marphys_Archive\\Data\\OSNAP\\RHADCP\\RHADCP_S55_data_JC238\\S200044A008_RHADCP_2020\\S200044A008_RHADCP_2020_A0_p.dat'

In [4]:
#  extract the header from each ensemble period
with open(file) as f:
    lines = f.readlines()
desired_lines = lines[0:1759135:57]

#  print to file then read again bu tis time concatenate the times
with open("header.txt", "w") as f:
    f.writelines(desired_lines)
    

In [5]:
# Month  Day Year Hour Minute Secon
timestamp=[]
with open('header.txt') as f:
    lines = f.readlines()       
    for x in lines:
            y=int(x.split()[2])
            m=int(x.split()[0])
            d=int(x.split()[1])
            h=int(x.split()[3])
            mn=int(x.split()[4])
            s=int(x.split()[5])
            d=datetime.datetime(y,m,d,h,mn,s)
            time=d.isoformat()
            timestamp.append(time)

In [13]:
with open(file) as f:
    lines = f.readlines()
    
start=1
stop=1
while stop<1759192:
    stop=start+56
    templines=lines[start:stop]
    start=stop+1
    templines
    if start==58:
        # get bin number coordinate
        bin=[]
        for x in templines:
            bin.append(x.split()[0])
        bin_num=np.array([int(x) for x in bin])
        
        # get first velocity measurement
        vele=[]
        veln=[]
        velu=[]
        for x in templines:
            vele.append(x.split()[2])
            veln.append(x.split()[3])
            velu.append(x.split()[4])
        vel_east=np.array([int(float(x)) for x in vele])
        vel_north=np.array([int(float(x)) for x in veln])
        vel_up=np.array([int(float(x)) for x in velu])
    else:
        vele=[]
        veln=[]
        velu=[]
        for x in templines:
            vele.append(x.split()[2])
            veln.append(x.split()[3])
            velu.append(x.split()[4])
        velex=np.array([int(float(x)) for x in vele])        
        velnx=np.array([int(float(x)) for x in veln])        
        velux=np.array([int(float(x)) for x in velu])        
        vel_east=np.c_[ vel_east, velex ]          
        vel_north=np.c_[ vel_north, velnx ]          
        vel_up=np.c_[ vel_up, velux ]  
        
        

KeyboardInterrupt: 

In [12]:
ds = xr.Dataset(
    {"velocity_east": (("bin", "time"), vel_east),
        "velocity_north": (("bin", "time"), vel_north),
        "velocity_up": (("bin", "time"), vel_up)
    },
    coords={
        "bin": bin_num,
        "time": timestamp,
    },
)

ValueError: conflicting sizes for dimension 'time': length 30863 on 'time' and length 3088 on 'velocity_east'